# Geocoding using Google API from CSV with latitude and longitude

In [1]:
!pip install -U googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.7.3-py3-none-any.whl size=40339 sha256=3594aa77c5e64a90f656756550c20610e5ab37415f454657c236a2149001cc7c
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/09/29/ed/84090c49f9572ea576ce9384d9ff02f7d4325c985710c53c4e
Successfully built googlemaps


In [2]:
import csv
import pandas as pd
import googlemaps
import folium

In [3]:
df_new = pd.read_csv('/resources/labs/DK Projects/2200addresses.csv')
df_new.head()

,name,fulladdress
0,John Doe 1,"11101 Gwynn Oaks Dr Unit 110, Raleigh, NC, 276..."
1,John Doe 2,"1340 Rio Falls Dr Apt 201, Raleigh, NC, 27614-..."
2,John Doe 3,"10716 Cardington Ln, Raleigh, NC, 27614-7052"
3,John Doe 4,"100 Carriage Trl, Raleigh, NC, 27614-9733"
4,John Doe 5,"1000 Chagford Way, Raleigh, NC, 27614-6513"


In [4]:
df_new.shape

(2200, 2)

In [5]:
gmaps_key = googlemaps.Client(key="youcan'thaveit")

In [6]:
add_1 = df_new['fulladdress'][0]
g = gmaps_key.geocode(add_1)
lat = g[0]["geometry"]["location"]["lat"]
long = g[0]["geometry"]["location"]["lng"]
print('Latitude: '+str(lat)+', Longitude: '+str(long))

Latitude: 35.9226859, Longitude: -78.568649


In [7]:
# geocode the entire dataframe:

def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]    
    return (lat)

df_new['lat'] = df_new['fulladdress'].apply(geocode)    


def geocode(add):
    g = gmaps_key.geocode(add)
    long = g[0]["geometry"]["location"]["lng"] 
    return (long)
    
   
df_new['long'] = df_new['fulladdress'].apply(geocode)

In [8]:
df_new.tail()

,name,fulladdress,lat,long
2195,John Doe 2196,"2727 Willow Pines Pl, Raleigh, NC, 27614-8085",35.920508,-78.563343
2196,John Doe 2197,"2728 Peachleaf St, Raleigh, NC, 27614-7965",35.958926,-78.555971
2197,John Doe 2198,"2728 Royal Forrest Dr, Raleigh, NC, 27614-7633",35.930605,-78.568890
2198,John Doe 2199,"2729 Falls River Ave, Raleigh, NC, 27614-9879",35.920001,-78.564477
2199,John Doe 2200,"2734 Cloud Mist Cir, Raleigh, NC, 27614-6637",35.928711,-78.563042


In [9]:
from folium.plugins import MarkerCluster

# center to the mean of all points
map = folium.Map(location=df_new[["lat", "long"]].mean().to_list(), zoom_start=14,)

# if the points are too close to each other, cluster them, create a cluster overlay with MarkerCluster
marker_cluster = MarkerCluster().add_to(map)

# draw the markers and assign popup and hover texts
# add the markers the the cluster layers so that they are automatically clustered
for i,r in df_new.iterrows():
    location = (r["lat"], r["long"])
    folium.Marker(location=location,
                      popup = r['name'],
                      tooltip=r['name'])\
    .add_to(marker_cluster)

# display the map
map

In [10]:
df_new.to_csv('/resources/labs/DK Projects/2200addresses-withlatlon.csv')